In [ ]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.4
    Uninstalling widgetsnbextension-3.6.4:
      Successfully uninstalled widgetsnbextension-3.6.4
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6

In [ ]:
import numpy as np

# first point cloud p
n = 1000
x = np.random.random(n) * 2 - 1
y = np.random.random(n) * 2 - 1
z = 4*( x * x + y * y)
p = np.c_[x, y, z]

def rotate_3d(p, yaw, pitch, roll):
    a = np.array([[+np.cos(yaw), -np.sin(yaw), 0],
                  [+np.sin(yaw), +np.cos(yaw), 0],
                  [0, 0, 1]])
    b = np.array([[+np.cos(pitch), 0, +np.sin(pitch)],
                  [0, 1, 0],
                  [-np.sin(pitch), 0, +np.cos(pitch)]])
    c = np.array([[1, 0, 0],
                  [0, +np.cos(roll), -np.sin(roll)],
                  [0, +np.sin(roll), +np.cos(roll)]])
    return a.dot(b.dot(c.dot(p.T))).T


# second point cloud q
q = rotate_3d(p, np.pi/4, np.pi/4, np.pi/4)
q += np.array([1.25, 1.50, 1.75])

In [ ]:
import open3d as o3d
import plotly.graph_objects as go
# Open3d point cloud objects

color1=np.tile(np.array([[1, 0.5, 0]]), (len(p), 1))
po = o3d.geometry.PointCloud()
po.points = o3d.utility.Vector3dVector(p)
po.colors =o3d.utility.Vector3dVector(color1)
points1=np.asarray(po.points)
color1= np.asarray(po.colors)

color2=np.tile(np.array([[0.5, 0, 1]]), (len(q), 1))
qo = o3d.geometry.PointCloud()
qo.points = o3d.utility.Vector3dVector(q)
qo.colors =o3d.utility.Vector3dVector(color2)
points2 = np.asarray(qo.points)
color2=np.asarray(qo.colors)

points=np.concatenate((points1,points2))
colors=np.concatenate((color1,color2))
fig = go.Figure(
  data=[
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1,color=colors)
)
],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
)
)
)
fig.show()


In [ ]:
def cross_cov(p, q, matches=None):
    n = p.shape[1]
    assert q.shape[1] == n, "dimension mismatch in cross_cov()"

    if matches is None:
        matches = [(i, i) for i in range(p.shape[0])]

    cov = np.zeros((n, n))
    matches = np.array(matches)
    pi = p[matches[:, 0]]
    qj = q[matches[:, 1]]
    cov = np.sum(pi[:, :, np.newaxis] * qj[:, np.newaxis, :], axis=0)

    return cov



In [ ]:
from scipy.spatial import cKDTree as kdtree
def transform(cc,pm,q,qm):
  u, s, v = np.linalg.svd(cc)
  r = u.dot(v)
  t = pm - r.dot(qm)
  q1 = r.dot(q.T).T + t

  return q1



In [ ]:
def show_pointcloud(p,q):
  color1=np.tile(np.array([[1, 0.5, 0]]), (len(p), 1))
  po = o3d.geometry.PointCloud()
  po.points = o3d.utility.Vector3dVector(p)
  po.colors =o3d.utility.Vector3dVector(color1)
  points1=np.asarray(po.points)
  color1= np.asarray(po.colors)

  color2=np.tile(np.array([[0.5, 0, 1]]), (len(q), 1))
  qo = o3d.geometry.PointCloud()
  qo.points = o3d.utility.Vector3dVector(q)
  qo.colors =o3d.utility.Vector3dVector(color2)
  points2 = np.asarray(qo.points)
  color2=np.asarray(qo.colors)

  points=np.concatenate((points1,points2))
  colors=np.concatenate((color1,color2))
  fig = go.Figure(
    data=[
      go.Scatter3d(
        x=points[:,0], y=points[:,1], z=points[:,2],
        mode='markers',
        marker=dict(size=1,color=colors)
  )
  ],
    layout=dict(
      scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
  )
  )
  )
  fig.show()


In [ ]:
def icp_svd(p, q, iterations=10):
    def matches(tr, q):
        dd, ii = tr.query(q, k=1, workers=-1)
        return [(ii[i], i) for i in range(ii.shape[0])]

    nq = q.shape[0]

    qs = np.empty((iterations, nq,3))
    pm = p.mean(axis=0)
    tr = kdtree(p - pm)
    qj = q.copy()
    for j in range(iterations):


        qm = qj.mean(axis=0)
        cc = cross_cov(p - pm, qj - qm, matches(tr, qj - qm))
        qj=transform(cc,pm,qj,qm)

        qs[j]=qj
        mse=np.mean((p - qj)**2)
        print('The iter {} and mean_square_error is {}'.format(j,mse))


    return qs

iterations=15
qs = icp_svd(p, q, iterations = iterations)

The iter 0 and mean_square_error is 0.5832651801963548
The iter 1 and mean_square_error is 0.29292712236785706
The iter 2 and mean_square_error is 0.15448956756690652
The iter 3 and mean_square_error is 0.08043375494561034
The iter 4 and mean_square_error is 0.050418964935900173
The iter 5 and mean_square_error is 0.02962654969170841
The iter 6 and mean_square_error is 0.017291118896106104
The iter 7 and mean_square_error is 0.011243591099487046
The iter 8 and mean_square_error is 0.0074231447037957626
The iter 9 and mean_square_error is 0.004784277074121699
The iter 10 and mean_square_error is 0.00275515182651834
The iter 11 and mean_square_error is 0.001117186530933548
The iter 12 and mean_square_error is 0.00012413891176076193
The iter 13 and mean_square_error is 2.1782526563793126e-07
The iter 14 and mean_square_error is 1.4449987696166573e-30


In [ ]:

for i in range(iterations):

  show_pointcloud(p,qs[i])